In [1]:
import numpy as np
import pandas as pd
import datetime
import requests
from bs4 import BeautifulSoup
import copy

In [2]:
df_val = pd.read_csv('../Data/Player Valuation/player_valuations.csv')
df_players = pd.read_csv('../Data/Player Valuation/players.csv')
df_clubs = pd.read_csv('../Data/Player Valuation/clubs.csv')


In [3]:
df_val.head()

,player_id,last_season,datetime,date,dateweek,market_value_in_eur,n,current_club_id,player_club_domestic_competition_id
0,3132,2013,2003-12-09 00:00:00,2003-12-09,2003-12-08,400000,1,126,TR1
1,6893,2012,2003-12-15 00:00:00,2003-12-15,2003-12-15,900000,1,984,GB1
2,10,2015,2004-10-04 00:00:00,2004-10-04,2004-10-04,7000000,1,398,IT1
3,26,2017,2004-10-04 00:00:00,2004-10-04,2004-10-04,1500000,1,16,L1
4,65,2015,2004-10-04 00:00:00,2004-10-04,2004-10-04,8000000,1,1091,GR1


In [4]:
df_players.head()

,player_id,first_name,last_name,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,foot,height_in_cm,market_value_in_eur,highest_market_value_in_eur,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name
0,598,Timo,Hildebrand,Timo Hildebrand,2014,24,timo-hildebrand,Germany,Worms,Germany,...,NaN,NaN,NaN,10000000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/timo-hildebran...,L1,Eintracht Frankfurt
1,670,Martin,Petrov,Martin Petrov,2012,714,martin-petrov,Bulgaria,Vratsa,Bulgaria,...,NaN,NaN,NaN,12000000.0,NaN,IFM,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/martin-petrov/...,ES1,RCD Espanyol Barcelona
2,1323,Martin,Amedick,Martin Amedick,2012,24,martin-amedick,Germany,Paderborn,Germany,...,NaN,NaN,NaN,2750000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/martin-amedick...,L1,Eintracht Frankfurt
3,3195,Jermaine,Pennant,Jermaine Pennant,2013,512,jermaine-pennant,England,Nottingham,England,...,right,173.0,NaN,10500000.0,NaN,Andrew Sky,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/jermaine-penna...,GB1,Stoke City
4,3259,Damien,Duff,Damien Duff,2013,931,damien-duff,Ireland,Ballyboden,Ireland,...,left,177.0,NaN,17000000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/damien-duff/pr...,GB1,Fulham FC


In [22]:
df_clubs.head()

,club_id,club_code,name,domestic_competition_id,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,url
0,105,sv-darmstadt-98,Sportverein Darmstadt 1898 e. V.,L1,NaN,30,26.4,11,36.7,1,Merck-Stadion am Böllenfalltor,17810,€-1.60m,NaN,2023,https://www.transfermarkt.co.uk/sv-darmstadt-9...
1,11127,ural-ekaterinburg,FK Ural Yekaterinburg,RU1,NaN,25,28.2,13,52.0,5,Yekaterinburg Arena,23000,€-770k,NaN,2023,https://www.transfermarkt.co.uk/ural-ekaterinb...
2,114,besiktas-istanbul,Beşiktaş Jimnastik Kulübü,TR1,NaN,32,27.4,16,50.0,13,Tüpraş Stadyumu,42590,€-14.50m,NaN,2023,https://www.transfermarkt.co.uk/besiktas-istan...
3,12,as-rom,Associazione Sportiva Roma,IT1,NaN,26,27.4,17,65.4,17,Olimpico di Roma,73261,+€65.20m,NaN,2023,https://www.transfermarkt.co.uk/as-rom/startse...
4,148,tottenham-hotspur,Tottenham Hotspur Football Club,GB1,NaN,29,26.2,21,72.4,22,Tottenham Hotspur Stadium,62062,€-130.40m,NaN,2023,https://www.transfermarkt.co.uk/tottenham-hots...


In [5]:
df_val_player = pd.merge(df_players, df_val, on='player_id')

In [6]:
df_val_player['date'] = pd.to_datetime(df_val_player['date'])
df_val_player.reset_index().set_index('name')
df_val_player.keys()

Index(['player_id', 'first_name', 'last_name', 'name', 'last_season_x',
       'current_club_id_x', 'player_code', 'country_of_birth', 'city_of_birth',
       'country_of_citizenship', 'date_of_birth', 'sub_position', 'position',
       'foot', 'height_in_cm', 'market_value_in_eur_x',
       'highest_market_value_in_eur', 'contract_expiration_date', 'agent_name',
       'image_url', 'url', 'current_club_domestic_competition_id',
       'current_club_name', 'last_season_y', 'datetime', 'date', 'dateweek',
       'market_value_in_eur_y', 'n', 'current_club_id_y',
       'player_club_domestic_competition_id'],
      dtype='object')

In [7]:
df_val_player.sort_values(by='date', ascending=False)

,player_id,first_name,last_name,name,last_season_x,current_club_id_x,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,current_club_domestic_competition_id,current_club_name,last_season_y,datetime,date,dateweek,market_value_in_eur_y,n,current_club_id_y,player_club_domestic_competition_id
74579,405676,Oliver,Bias,Oliver Bias,2018,23826,oliver-bias,Germany,Johanngeorgenstadt,Philippines,...,L1,RB Leipzig,2018,2023-12-19 00:00:00,2023-12-19,2023-12-18,50000,1,23826,L1
430420,718115,NaN,Zini,Zini,2023,2441,zini,Angola,Cazenga,Angola,...,GR1,AEK Athens,2023,2023-09-21 00:00:00,2023-09-21,2023-09-18,300000,1,2441,GR1
276409,96790,Sebastian,Mladen,Sebastian Mladen,2023,6418,sebastian-mladen,Romania,Calafat,Romania,...,GR1,Panetolikos GFS,2023,2023-09-21 00:00:00,2023-09-21,2023-09-18,500000,1,6418,GR1
126125,335725,NaN,Shapi Suleymanov,Shapi Suleymanov,2023,605,shapi-suleymanov,Russia,Makhachkala,Russia,...,GR1,Aris Thessaloniki,2023,2023-09-21 00:00:00,2023-09-21,2023-09-18,2000000,1,605,GR1
223138,88348,Alexandros,Paschalakis,Alexandros Paschalakis,2023,683,alexandros-paschalakis,Greece,Athens,Greece,...,GR1,Olympiacos Piraeus,2023,2023-09-21 00:00:00,2023-09-21,2023-09-18,800000,1,683,GR1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268490,12503,Souleymane,Diawara,Souleymane Diawara,2014,417,souleymane-diawara,Senegal,Gabou,Senegal,...,FR1,OGC Nice,2014,2004-10-04 00:00:00,2004-10-04,2004-10-04,2500000,1,417,FR1
29801,9694,Efthymios,Kouloucheris,Efthymios Kouloucheris,2013,169,efthymios-kouloucheris,Greece,Karditsa,Greece,...,GR1,Panionios Athens,2013,2004-10-04 00:00:00,2004-10-04,2004-10-04,300000,1,169,GR1
268513,12665,Keith,Lasley,Keith Lasley,2016,987,keith-lasley,Scotland,Glasgow,Scotland,...,SC1,Motherwell FC,2016,2004-10-04 00:00:00,2004-10-04,2004-10-04,50000,1,987,SC1
105111,6893,Gabriel,Tamas,Gabriel Tamas,2012,984,gabriel-tamas,Romania,Brașov,Romania,...,GB1,West Bromwich Albion,2012,2003-12-15 00:00:00,2003-12-15,2003-12-15,900000,1,984,GB1


In [8]:
new_date = datetime.datetime(2023, 1, 1)
df_val_player.loc[(df_val_player['name'] == 'Victor Osimhen') & (df_val_player['date'] > new_date)].sort_values(by='date', ascending=False)


,player_id,first_name,last_name,name,last_season_x,current_club_id_x,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,current_club_domestic_competition_id,current_club_name,last_season_y,datetime,date,dateweek,market_value_in_eur_y,n,current_club_id_y,player_club_domestic_competition_id
435575,401923,Victor,Osimhen,Victor Osimhen,2023,6195,victor-osimhen,Nigeria,Lagos,Nigeria,...,IT1,SSC Napoli,2023,2023-06-15 00:00:00,2023-06-15,2023-06-12,120000000,1,6195,IT1
435574,401923,Victor,Osimhen,Victor Osimhen,2023,6195,victor-osimhen,Nigeria,Lagos,Nigeria,...,IT1,SSC Napoli,2023,2023-03-21 00:00:00,2023-03-21,2023-03-20,100000000,1,6195,IT1


In [9]:
def nan_values(df: pd.DataFrame):
    cols = df.columns.values.tolist()
    rows = df.index.values.tolist()
    for col in cols:
        if df[col].isna().sum() == 0:
            pass
        else:
            print(f"for {col}: {df[col].isna().sum()} NaN values")


In [10]:
checkdate = datetime.datetime(2023, 6, 1)
df_val_player_new = df_val_player[['player_id', 'name', 'market_value_in_eur_y', 'date']]
df_val_player_new = df_val_player_new.loc[(df_val_player['date'] > checkdate)]

In [11]:
df_val_player_new.loc[df_val_player_new['name'] == 'Kylian Mbappé']

,player_id,name,market_value_in_eur_y,date
73019,342229,Kylian Mbappé,180000000,2023-06-27


In [15]:
df_val_player_new.loc[df_val_player_new['market_value_in_eur_y'] >= 100]['name'].to_list()

['Karim Benzema',
 'Jesper Hansen',
 'Óscar Trejo',
 'Jessy Moulin',
 'Steven Fletcher',
 'Vítor Gomes',
 'Joe Hart',
 'Jan Vertonghen',
 'Adam Legzdins',
 'Antonio Adán',
 'Federico Fazio',
 'Ángel Di María',
 'Samuel Di Carmine',
 'André Ayew',
 'Aleksandar Mitrović',
 'Fabian Holland',
 "Danilo D'Ambrosio",
 'Alexandre Oukidja',
 'Sebastian Rudy',
 'Vadym Milko',
 'David Alaba',
 'Gernot Trauner',
 'Victor Moses',
 'Anastasios Avlonitis',
 'Maxime Le Marchand',
 'Roberto Soriano',
 'Bernardo Espinosa',
 'Pedro',
 'Juan Cala',
 'Mathias Gehrt',
 'David Simão',
 'Oliver Lund',
 'Nemanja Matic',
 'Eiji Kawashima',
 'Anthony Moris',
 'Ali Crawford',
 'Antonio Jakolis',
 'Maya Yoshida',
 'Nathaniel Clyne',
 'Roman Akbashev',
 'Antonio Rüdiger',
 'Alfred Finnbogason',
 'Lukas Van Eenoo',
 'Mats Rits',
 'Volodymyr Tanchyk',
 'Viktor Fischer',
 'Andrea Bertolacci',
 'Jeffrey Gouweleeuw',
 'Stefan Savic',
 'Koen Casteels',
 'Fredrik Gulbrandsen',
 'Jed Steer',
 'Emil Forsberg',
 'Koray Günte

In [12]:
df_val_player_new.to_csv('../Data/Player Valuation/simple_valuations.csv', index=False)

In [30]:
nan_values(df_val_player_new)

In [15]:
df_player_defense       = pd.read_csv('../Data/FIFA World Cup 2022 Player Data/player_defense.csv', delimiter=',')
df_player_gca           = pd.read_csv('../Data/FIFA World Cup 2022 Player Data/player_gca.csv', delimiter=',')
df_player_keepers       = pd.read_csv('../Data/FIFA World Cup 2022 Player Data/player_keepers.csv', delimiter=',')
df_player_keepersadv    = pd.read_csv('../Data/FIFA World Cup 2022 Player Data/player_keepersadv.csv', delimiter=',')
df_player_misc          = pd.read_csv('../Data/FIFA World Cup 2022 Player Data/player_misc.csv', delimiter=',')
df_player_passing       = pd.read_csv('../Data/FIFA World Cup 2022 Player Data/player_passing.csv', delimiter=',')
df_player_passing_types = pd.read_csv('../Data/FIFA World Cup 2022 Player Data/player_passing_types.csv', delimiter=',')
df_player_playingtime   = pd.read_csv('../Data/FIFA World Cup 2022 Player Data/player_playingtime.csv', delimiter=',')
df_player_possession    = pd.read_csv('../Data/FIFA World Cup 2022 Player Data/player_possession.csv', delimiter=',')
df_player_shooting      = pd.read_csv('../Data/FIFA World Cup 2022 Player Data/player_shooting.csv', delimiter=',')
df_player_stats         = pd.read_csv('../Data/FIFA World Cup 2022 Player Data/player_stats.csv', delimiter=',')

In [2]:
url_stats = 'https://fbref.com/en/comps/Big5/2022-2023/stats/players/2022-2023-Big-5-European-Leagues-Stats'
url_mics  = 'https://fbref.com/en/comps/Big5/2022-2023/misc/players/2022-2023-Big-5-European-Leagues-Stats' 
url_shooting = 'https://fbref.com/en/comps/Big5/2022-2023/shooting/players/2022-2023-Big-5-European-Leagues-Stats'
url_passing  = 'https://fbref.com/en/comps/Big5/2022-2023/passing/players/2022-2023-Big-5-European-Leagues-Stats'
url_defense = 'https://fbref.com/en/comps/Big5/2022-2023/defense/players/2022-2023-Big-5-European-Leagues-Stats'
url_gca = 'https://fbref.com/en/comps/Big5/2022-2023/gca/players/2022-2023-Big-5-European-Leagues-Stats'
url_keepersadv = 'https://fbref.com/en/comps/Big5/2022-2023/keepersadv/players/2022-2023-Big-5-European-Leagues-Stats'
url_keepers = 'https://fbref.com/en/comps/Big5/2022-2023/keepers/players/2022-2023-Big-5-European-Leagues-Stats'
url_playingtime = 'https://fbref.com/en/comps/Big5/2022-2023/playingtime/players/2022-2023-Big-5-European-Leagues-Stats'
url_possession = 'https://fbref.com/en/comps/Big5/2022-2023/possession/players/2022-2023-Big-5-European-Leagues-Stats'
url_passingtypes = 'https://fbref.com/en/comps/Big5/2022-2023/passing_types/players/2022-2023-Big-5-European-Leagues-Stats'


In [13]:
def scrape_player_data(url: str):
    df_temp = pd.read_html(url)[0]
    df_temp.columns = [' '.join(col).strip() for col in df_temp.columns]

    df_temp = df_temp.reset_index(drop=True)
    return df_temp

In [14]:
df_stats = scrape_player_data(url_stats)
df_misc = scrape_player_data(url_mics)
df_passing = scrape_player_data(url_passing)
df_shooting = scrape_player_data(url_passing)
df_defense = scrape_player_data(url_defense)
df_gca = scrape_player_data(url_gca)
df_keepers = scrape_player_data(url_keepers)
df_keepersadv = scrape_player_data(url_keepersadv)
df_playingtime = scrape_player_data(url_playingtime)
df_possesion = scrape_player_data(url_possession)
df_passingtypes = scrape_player_data(url_passingtypes)

In [15]:
df_stats

,Unnamed: 0_level_0 Rk,Unnamed: 1_level_0 Player,Unnamed: 2_level_0 Nation,Unnamed: 3_level_0 Pos,Unnamed: 4_level_0 Squad,Unnamed: 5_level_0 Comp,Unnamed: 6_level_0 Age,Unnamed: 7_level_0 Born,Playing Time MP,Playing Time Starts,...,Per 90 Minutes Ast,Per 90 Minutes G+A,Per 90 Minutes G-PK,Per 90 Minutes G+A-PK,Per 90 Minutes xG,Per 90 Minutes xAG,Per 90 Minutes xG+xAG,Per 90 Minutes npxG,Per 90 Minutes npxG+xAG,Unnamed: 37_level_0 Matches
0,1,Brenden Aaronson,us USA,"MF,FW",Leeds United,eng Premier League,21,2000,36,28,...,0.11,0.15,0.04,0.15,0.15,0.16,0.31,0.15,0.31,Matches
1,2,Paxten Aaronson,us USA,"MF,DF",Eint Frankfurt,de Bundesliga,18,2003,7,0,...,0.00,0.00,0.00,0.00,0.09,0.03,0.11,0.09,0.11,Matches
2,3,James Abankwah,ie IRL,DF,Udinese,it Serie A,18,2004,2,1,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches
3,4,George Abbott,eng ENG,MF,Tottenham,eng Premier League,16,2005,1,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches
4,5,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,34,1987,37,37,...,0.05,0.08,0.03,0.08,0.06,0.03,0.09,0.06,0.09,Matches
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999,2885,Martin Ødegaard,no NOR,MF,Arsenal,eng Premier League,23,1998,37,37,...,0.20,0.63,0.43,0.63,0.29,0.23,0.52,0.29,0.52,Matches
3000,2886,Milan Đurić,ba BIH,FW,Hellas Verona,it Serie A,32,1990,28,14,...,0.07,0.13,0.07,0.13,0.09,0.08,0.18,0.09,0.18,Matches
3001,2887,Filip Đuričić,rs SRB,"MF,FW",Sampdoria,it Serie A,30,1992,32,27,...,0.00,0.12,0.12,0.12,0.11,0.08,0.19,0.11,0.19,Matches
3002,2888,Blanco,NaN,MF,Cádiz,es La Liga,22,2000,3,2,...,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,Matches


In [18]:
df_shooting.loc[df_shooting['Unnamed: 1_level_0 Player'] == 'Nacho']

,Unnamed: 0_level_0 Rk,Unnamed: 1_level_0 Player,Unnamed: 2_level_0 Nation,Unnamed: 3_level_0 Pos,Unnamed: 4_level_0 Squad,Unnamed: 5_level_0 Comp,Unnamed: 6_level_0 Age,Unnamed: 7_level_0 Born,Unnamed: 8_level_0 90s,Total Cmp,...,Unnamed: 23_level_0 Ast,Unnamed: 24_level_0 xAG,Expected xA,Expected A-xAG,Unnamed: 27_level_0 KP,Unnamed: 28_level_0 1/3,Unnamed: 29_level_0 PPA,Unnamed: 30_level_0 CrsPA,Unnamed: 31_level_0 PrgP,Unnamed: 32_level_0 Matches
1965,1891,Nacho,es ESP,DF,Real Madrid,es La Liga,32,1990,18.2,1085,...,1,0.9,0.7,+0.1,12,79,8,1,75,Matches


In [37]:
df_defense

,Unnamed: 0_level_0 Rk,Unnamed: 1_level_0 Player,Unnamed: 2_level_0 Nation,Unnamed: 3_level_0 Pos,Unnamed: 4_level_0 Squad,Unnamed: 5_level_0 Comp,Unnamed: 6_level_0 Age,Unnamed: 7_level_0 Born,Unnamed: 8_level_0 90s,Tackles Tkl,...,Challenges Tkl%,Challenges Lost,Blocks Blocks,Blocks Sh,Blocks Pass,Unnamed: 21_level_0 Int,Unnamed: 22_level_0 Tkl+Int,Unnamed: 23_level_0 Clr,Unnamed: 24_level_0 Err,Unnamed: 25_level_0 Matches
0,1,Brenden Aaronson,us USA,"MF,FW",Leeds United,eng Premier League,21,2000,26.4,45,...,32.6,31,43,3,40,5,50,6,1,Matches
1,2,Paxten Aaronson,us USA,"MF,DF",Eint Frankfurt,de Bundesliga,18,2003,1.9,6,...,66.7,1,2,1,1,0,6,2,0,Matches
2,3,James Abankwah,ie IRL,DF,Udinese,it Serie A,18,2004,0.7,1,...,NaN,0,4,2,2,0,1,5,0,Matches
3,4,George Abbott,eng ENG,MF,Tottenham,eng Premier League,16,2005,0.0,0,...,NaN,0,0,0,0,1,1,0,0,Matches
4,5,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,34,1987,37.0,82,...,75.0,13,65,27,38,64,146,116,1,Matches
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999,2885,Martin Ødegaard,no NOR,MF,Arsenal,eng Premier League,23,1998,34.7,36,...,29.8,33,20,5,15,6,42,12,1,Matches
3000,2886,Milan Đurić,ba BIH,FW,Hellas Verona,it Serie A,32,1990,15.2,2,...,33.3,2,7,4,3,1,3,24,0,Matches
3001,2887,Filip Đuričić,rs SRB,"MF,FW",Sampdoria,it Serie A,30,1992,24.1,43,...,43.8,18,19,6,13,7,50,8,0,Matches
3002,2888,Blanco,NaN,MF,Cádiz,es La Liga,22,2000,1.8,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,Matches


In [47]:
list_dfs = [df_misc, df_stats, df_shooting, df_defense, df_gca, df_keepers, df_keepersadv, df_passing, df_passingtypes, df_possesion, df_playingtime]
df_merged = pd.concat(list_dfs, axis=1, join='inner')